In [36]:
# basics
import os
import utils
import numpy as np
from tqdm.notebook import tqdm 


# torch
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import lightning as L




# custom modules
from data_acquisition import DataHandler
from data_preparation import apply_preprocessing_pipeline


# Configure logging for the pipeline
logger = utils.setup_logger(level='ERROR')

In [2]:
cities = ['London', 'CapeTown', 'Hamburg', 'Johannesburg', 'London', 'Montreal', 'Paris', 'Seoul', 'Singapore', 'Sydney']

datahandler = DataHandler(logger)


In [3]:
# load images and mask for all specified cites

import os
images = []
sparse_masks=[]
dense_masks=[]

for city in tqdm(cities):
    buildings = None
    if not os.path.exists(f'data/{city}/building_mask_dense.tif'):
        print("loading local buildings")
        buildings = datahandler.get_buildings(city)
    images.append(datahandler.get_satellite_image(city))
    sparse_masks.append(datahandler.get_building_mask(city, all_touched=False, loaded_buildings=buildings))
    dense_masks.append(datahandler.get_building_mask(city, all_touched=True, loaded_buildings=buildings))

  0%|          | 0/4 [00:00<?, ?it/s]

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as 

In [4]:
masks = sparse_masks


In [5]:
# apply training pipeline
# TODO make train test split consistent so we can train with multiple sizes, dont know if there is an advantage though
train_loader, test_loader , validation_loader= apply_preprocessing_pipeline(images, masks, patch_size = 128, test_ratio= 0.2,validation_ratio=0.2, batch_size = 64, show_validation_of_split=True,city_names=cities)

✓ Differences of train and test set is below 10% on mean, std, min and max across all input bands
✓ Differences of train and validation set is below 10% on mean, std, min and max across all input bands

Comparison of distribution of masks:
                   mean  median          std  10th percentile  90th percentile
Train       600.190379   116.0  1032.131645              0.0           2023.6
Test        593.828725   114.5  1028.243032              0.0           2054.9
Validation  592.890937   103.5  1025.142817              0.0           2072.5

Comparison of cities the data in the differen sets originates from:
               London  CapeTown   Hamburg  Johannesburg
Train       0.334186  0.142272  0.143296      0.380246
Test        0.319508  0.150538  0.146697      0.383257
Validation  0.311828  0.159754  0.137481      0.390937


In [11]:
sample = next(iter(train_loader))   
sample.shape, type(sample), sample.dtype

(torch.Size([64, 7, 128, 128]), torch.Tensor, torch.uint16)

In [27]:
class convNetSimple(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
                nn.Conv2d(6, 32, kernel_size=3, padding=1), nn.ReLU(),
                nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
                nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
                nn.Conv2d(128, 1, kernel_size=1, padding=0),
                nn.Sigmoid())
    
    def forward(self, x):
        return self.model(x)
    
class LitNet(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = nn.BCELoss()

    def training_step(self, batch, batch_idx):
        x, y = batch[:,:-1], batch[:,-1]
        outs = self.model(x.float())
        loss = self.loss(outs, y.unsqueeze(1).float())
        self.log("train_loss", value=loss, on_step=True, on_epoch=True, logger=True, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch[:,:-1], batch[:,-1]
        outs = self.model(x.float())
        loss = self.loss(outs, y.unsqueeze(1).float())
        
        values = {
            "test_loss": loss,
        }
        self.log_dict(values, on_epoch=True, on_step=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def forward(self, x):
        return self.model(x)


### Load Model

In [28]:
# load trained model
convmodel = LitNet(convNetSimple())
model_dict = torch.load("models/lightning_logs/version_1/checkpoints/epoch=39-step=7000.ckpt", map_location=torch.device('cpu'))
convmodel.load_state_dict(model_dict['state_dict'])

<All keys matched successfully>

In [42]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint


L.seed_everything(42)
convmodel = LitNet(convNetSimple())
trainer = L.Trainer(
    default_root_dir="models",
    # callbacks=[
    #     EarlyStopping(
    #         monitor="val_loss",
    #         mode="min",
    #         patience=10,
    #     )
    #     ModelCheckpoint(
    #         monitor="val_loss",
    #         mode="min",
    #         save_top_k=2,
    #         dirpath="models",
    #         filename="best_model"
    #     )
    # ]
    # val_check_interval=1,
    fast_dev_run=False,
    # num_sanity_val_steps=2,
    max_epochs=40,
    log_every_n_steps=20,
)

# training
trainer.fit(convmodel, 
    train_dataloaders=train_loader,
    # val_dataloaders=val_loader   #### HIER brauchen wir noch einen validation loader
)


# testing

# hier könnte man noch das beste model laden, wenn wir ein Val dataset haben.
# best_model = LitModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
# trainer.test(
#     best_model,
#     dataloaders=test_loader
# )


trainer.test(
    convmodel,
    dataloaders=test_loader
)


INFO: Seed set to 42
2024-06-28 23:29:49,796 - lightning.fabric.utilities.seed - INFO - seed_everything - Seed set to 42
2024-06-28 23:29:49,796 - lightning.fabric.utilities.seed - INFO - seed_everything - Seed set to 42
INFO: GPU available: True (cuda), used: True
2024-06-28 23:29:49,816 - lightning.pytorch.utilities.rank_zero - INFO - _info - GPU available: True (cuda), used: True
2024-06-28 23:29:49,816 - lightning.pytorch.utilities.rank_zero - INFO - _info - GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
2024-06-28 23:29:49,818 - lightning.pytorch.utilities.rank_zero - INFO - _info - TPU available: False, using: 0 TPU cores
2024-06-28 23:29:49,818 - lightning.pytorch.utilities.rank_zero - INFO - _info - TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
2024-06-28 23:29:49,819 - lightning.pytorch.utilities.rank_zero - INFO - _info - HPU available: False, using: 0 HPUs
2024-06-28 23:29:49,819 - lightning.pytorch.

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=40` reached.
2024-06-28 23:47:55,487 - lightning.pytorch.utilities.rank_zero - INFO - _info - `Trainer.fit` stopped: `max_epochs=40` reached.
2024-06-28 23:47:55,487 - lightning.pytorch.utilities.rank_zero - INFO - _info - `Trainer.fit` stopped: `max_epochs=40` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2024-06-28 23:47:55,519 - lightning.pytorch.accelerators.cuda - INFO - set_nvidia_flags - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2024-06-28 23:47:55,519 - lightning.pytorch.accelerators.cuda - INFO - set_nvidia_flags - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jlb/Dev/architecture-of-ml-systems/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=20` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch         9.542298316955566
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 9.542298316955566}]

In [ ]:
# # Instantiate the model, loss function, and optimizer
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# # Training loop
# num_epochs = 50

# model.train()
# for epoch in tqdm(range(num_epochs)):
#     for batch in train_loader:
#         # splid in inputs and labels
#         inputs = batch[:,:-1].to(torch.float32)
#         labels = batch[:,-1, np.newaxis].to(torch.float32)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward pass
#         outputs = model(inputs)

#         # calculate loss
#         loss = criterion(outputs, labels)

#         # write to tensorboard
#         writer.add_scalar("Loss/train", loss, epoch)

#         # backward pass
#         loss.backward()

#         # optimizer step
#         optimizer.step()
    
# writer.flush()

## Save Model

In [30]:
# import os

# os.makedirs("saved_models", exist_ok=True)
# torch.save(model.state_dict(), "saved_models/model1")

# Evaluation

In [52]:
# predict on test set
t  = torch.Tensor(test_loader.dataset)

# splid in inputs and labels
test_inputs = t[:,:-1]#.to(torch.float32)
test_labels = t[:,-1, np.newaxis]#.to(torch.float32)

test_results = convmodel(test_inputs).detach()

# Look at sums, to check if model only predicts zeros
print("Sum of test results: ", test_results.sum())
print("But it should be closer to: ", test_labels.sum())


# # see how many percnet where predicted right
threshold = 0.5
((test_results>threshold)==test_labels).sum()/np.prod(test_labels.shape)

tensor(0.9638)

In [ ]:
# from sklearn.metrics import RocCurveDisplay

# RocCurveDisplay.from_predictions(
#    test_labels.flatten(), test_results.flatten())

# Evaluation

In [31]:
# t  = torch.Tensor(test_loader.dataset)

# # splid in inputs and labels
# test_inputs = t[:,:-1]#.to(torch.float32)
# test_labels = t[:,-1, np.newaxis]#.to(torch.float32)

# test_results = model(test_inputs).detach()





# Download

In [ ]:


buildings = []
sat_images = []
building_masks = []

for city in cities: 
    buildings.append(datahandler.get_buildings(city))
    sat_images.append(datahandler.get_satellite_image(city))
    building_masks.append(datahandler.get_building_mask(city))

# Plot the expected results for the first city 
datahandler.plot(city[0])

In [ ]:
import data_preparation

for city in cities:
    data_preparation.create_tensor(city)

# Download

In [ ]:
# Download 

for city in cities: 
    sat_image = datahandler.get_satellite_image(city)
    mask = datahandler.get_building_mask(city)

# Plot the expected results for the first city 
datahandler.plot(city[0])